In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)


In [ ]:
df = pd.read_csv("../data/processed/sentiment_labeled_data.csv")
df["time"] = pd.to_datetime(df["time"])

print(df.shape)
display(df.head())


In [ ]:
df["sentiment_code"] = df["Classification"].map({
    "Fear": 0,
    "Greed": 1
})

df["regime"] = df["sentiment_code"].map({
    0: "Fear Regime",
    1: "Greed Regime"
})


In [ ]:
df = df.sort_values("time")
df["prev_regime"] = df["regime"].shift(1)
df["regime_transition"] = df["prev_regime"] + " → " + df["regime"]


In [ ]:
transition_matrix = pd.crosstab(df["prev_regime"], df["regime"])
display(transition_matrix)


In [ ]:
df["regime_change"] = (df["regime"] != df["prev_regime"]).astype(int)

persistence = df.groupby("regime")["regime_change"].mean()
display(persistence)


In [ ]:
regime_performance = df.groupby("regime").agg(
    avg_pnl=("normalized_pnl", "mean"),
    win_rate=("is_profitable", "mean"),
    avg_risk=("risk_score", "mean"),
    avg_leverage=("leverage", "mean")
)

display(regime_performance)


In [ ]:
plt.figure(figsize=(10,4))
sns.boxplot(data=df, x="regime", y="normalized_pnl")
plt.title("PnL Distribution by Market Regime")
plt.show()

plt.figure(figsize=(10,4))
sns.boxplot(data=df, x="regime", y="leverage")
plt.title("Leverage Distribution by Market Regime")
plt.show()


In [ ]:
behavioral = df.groupby("regime").agg(
    avg_aggression=("aggression_score", "mean"),
    avg_discipline=("discipline_score", "mean"),
    overtrading=("overtrading_score", "mean")
)

display(behavioral)


In [ ]:
regime_performance.to_csv("../data/processed/regime_performance.csv")
transition_matrix.to_csv("../data/processed/regime_transition_matrix.csv")
behavioral.to_csv("../data/processed/regime_behavioral_metrics.csv")

print("✅ Regime analytics saved.")
